## Notebook para la consolidación de datos

Este notebook es el que realiza toda la consolidación  y preprocesamiento de las variables de salida (costos y gastos de venta) y de las variables de entrada (variables macroeconómicas) que serán usadas por el programa de R.

In [1]:
### Importar librerías
import pandas as pd

### Parte 1: variables de salida (costos y gastos de venta)

Se descargaron de la pagina http://pie.supersociedades.gov.co los archivos que tienen la información de las variables de salida (costos y gastos de ventas),
Éstos se guardaron en la carpeta /PreparacionDatos/Data.

Se descargan los de los años 2015 a 2018. El análisis de R se enfoca solo en los años de 2016 a 2018, pero se necesita 2015 para las variables en diferencias de 2016.

In [2]:
### Leer dataframe de cada year (la de los ERI que es la que tiene los costos y gastos de ventas)
dat2015 = pd.read_csv('Data/EstadoResultadosIntegrales(ERI)2015.txt', sep='¬')
dat2016 = pd.read_csv('Data/EstadoResultadosIntegrales(ERI)2016.txt', sep='¬')
dat2017 = pd.read_excel('Data/NIIF Plenas Individuales2017.xlsx', sheet_name='Estado de Resultados Integrales')
dat2018 = pd.read_excel('Data/NIIF Plenas Individuales2018.xlsx', sheet_name='ERI')

### Corregir el nombre de la columna NIT de 2018 para que quede igual a la de los demas
dat2018 = dat2018.rename(columns={"Nit": "NIT"})


C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  
C:\Users\nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


##### Dejar solo las empresas de la clasifiacion industrial internacion uniformeseleccionada. En este caso, usaremos solo empresas relacionadas con "construcción".

Se dejan solo las empresas que tengan la palabra 'construcción' en su clasificacion uniforme (nos basaremos en las del 2018 para obtener los NIT de esas empresas, 
y luego filtraremos los dataframes de years pasados para dejar solo esos NIT).

NOTA: solo se tienen en cuenta las empresas que pertenecen a los archivo de Plenas y de Individuales.

In [3]:

### Leer archivo guia donde esta la clasificacion uniforme para saber que NITs conservar
guia_empresas =  pd.read_excel('Data/NIIF Plenas Individuales2018.xlsx', sheet_name='Caratula').rename(columns={"Nit": "NIT"})



### Pasar todo a minuscula la columna de clasificacion
guia_empresas['clasificacion'] = guia_empresas['Clasificación Industrial Internacional Uniforme Versión 4 A.C'].str.lower()

### Dejar solo las que tengan la palabra "construcción" en su clasificacion
empresas_usar = guia_empresas[guia_empresas['clasificacion'].str.contains("construcción")]

In [4]:

### Extraer NIT de las empresas a incluir
NIT_empresas = list(empresas_usar['NIT'].values)


### Verificar cuales de esos NIT aparecen en los 4 years (2015 a 2018)
nit_todos_periodos = []
for nit in NIT_empresas:
    if nit in dat2015['NIT'].values and nit in dat2016['NIT'].values and nit in dat2017['NIT'].values and nit in dat2018['NIT'].values:
        nit_todos_periodos.append(nit)



#### Agregar la columna year a cada datafame
dat2015['Year'] = 2015
dat2016['Year'] = 2016
dat2017['Year'] = 2017
dat2018['Year'] = 2018

### Gastos de ventas de 2017 se llamaba Costos de distribución
dat2017['Gastos de ventas'] = dat2017['Costos de distribución']



### Dejar solo los periodos actuales en 2017 y 2018
dat2017 = dat2017[dat2017['PERIODO']=='2017']
dat2018 = dat2018[dat2018['Periodo']=='Periodo Actual']


In [5]:

#### Filtrar los dataframe para dejar solo los nit indicados y
### conservar solo las columnqs de Nit, gatos de ventas y costo de ventas
dat_2015_f = dat2015[dat2015['NIT'].isin(nit_todos_periodos)][['NIT','Year','Costo de ventas', 'Gastos de ventas']]
dat_2016_f = dat2016[dat2016['NIT'].isin(nit_todos_periodos)][['NIT','Year','Costo de ventas', 'Gastos de ventas']]
dat_2017_f = dat2017[dat2017['NIT'].isin(nit_todos_periodos)][['NIT','Year','Costo de ventas', 'Gastos de ventas']]
dat_2018_f = dat2018[dat2018['NIT'].isin(nit_todos_periodos)][['NIT','Year','Costo de ventas', 'Gastos de ventas']]


### Quitar duplicados de 2015 (quedarse con el primero)
dat_2015_f.drop_duplicates(inplace=True, subset=['NIT']) 

### Ordenar cada dataframe por Nit
dat_2015_auxi = dat_2015_f.sort_values(by='NIT').reset_index(drop=True)
dat_2016_auxi = dat_2016_f.sort_values(by='NIT').reset_index(drop=True)
dat_2017_auxi = dat_2017_f.sort_values(by='NIT').reset_index(drop=True)
dat_2018_auxi = dat_2018_f.sort_values(by='NIT').reset_index(drop=True)

Se usa la variación porcentual anual como la variable de salida. Por ejemplo, la fórmula para la variación porcentual anual de costos de venta sería esta:

$ CostoDeVentasDif(t) = \dfrac{CostoDeVentas(t) – CostoDeVentas(t-1)}{|CostoDeVentas(t-1)|} $

Se usa una fórmula análoga para gastos de venta.

NOTA: se acalara que en el análisis de R se aplica una transformación de logaritmo tanto a saldias como entradas, pero en este notebook de Python solo se llega hasta la parte de dejar las salidas y muchas de las entradas como variaciones porcentuales anuales.

In [6]:
### Calular valores en variación porcentual anual de costo de ventas y gastos de ventas
dat_2016_auxi['Costo de ventas_dif'] = (dat_2016_auxi['Costo de ventas'] - dat_2015_auxi['Costo de ventas'])/dat_2015_auxi['Costo de ventas']
dat_2017_auxi['Costo de ventas_dif'] = (dat_2017_auxi['Costo de ventas'] - dat_2016_auxi['Costo de ventas'])/dat_2016_auxi['Costo de ventas']
dat_2018_auxi['Costo de ventas_dif'] = (dat_2018_auxi['Costo de ventas'] - dat_2017_auxi['Costo de ventas'])/dat_2017_auxi['Costo de ventas']
dat_2016_auxi['Gastos de ventas_dif'] = (dat_2016_auxi['Gastos de ventas'] - dat_2015_auxi['Gastos de ventas'])/dat_2015_auxi['Gastos de ventas']
dat_2017_auxi['Gastos de ventas_dif'] = (dat_2017_auxi['Gastos de ventas'] - dat_2016_auxi['Gastos de ventas'])/dat_2016_auxi['Gastos de ventas']
dat_2018_auxi['Gastos de ventas_dif'] = (dat_2018_auxi['Gastos de ventas'] - dat_2017_auxi['Gastos de ventas'])/dat_2017_auxi['Gastos de ventas']


In [7]:
### Identificar empresas con un NaN en alguna de sus filas
### Eliminar las empresas que tengan al menos un NaN en algun year
nans2016 = dat_2016_auxi[dat_2016_auxi.isnull().any(axis=1)].index
nans2017 = dat_2017_auxi[dat_2017_auxi.isnull().any(axis=1)].index
nans2018 = dat_2018_auxi[dat_2018_auxi.isnull().any(axis=1)].index

#### Tambien las que tengan un 0 en costo o gasto (esto debe ser un error)
ceros2016 = dat_2016_auxi[(dat_2016_auxi==0).any(axis=1)].index
ceros2017 = dat_2017_auxi[dat_2017_auxi.isnull().any(axis=1)].index
ceros2018 = dat_2018_auxi[(dat_2018_auxi==0).any(axis=1)].index


### Indices a remover
indices_remover = list(set(nans2016) | set(nans2017) | set(nans2018) | set(ceros2016) | set(ceros2018))  ## falta ceros 2017 

### Indices a conservar
indices_conservar = list(set(range(len(dat_2018_auxi))) - set(indices_remover))


### Dejar solo empresas sin ningun NaN
dat_2016_fin = dat_2016_auxi.iloc[indices_conservar].reset_index(drop=True)
dat_2017_fin = dat_2017_auxi.iloc[indices_conservar].reset_index(drop=True)
dat_2018_fin = dat_2018_auxi.iloc[indices_conservar].reset_index(drop=True)


In [8]:

#### Crear dataframe con las variables de salida

### Juntar las de 2016, 2017 y 2017 (version fin) en un solo dataframe
salidas_temp = pd.concat([dat_2016_fin, dat_2017_fin, dat_2018_fin]).reset_index(drop=True)




Guardar las variables de salida en un csv.

In [9]:
### Guardar csv con las variables de salida
salidas_temp.to_csv('Auxi/Costos_Gastos_ventas.csv', index=False)


Más adelante usaremos de nuevo este dataframe. Por ahora, procesaremos las variables explicativas.

### Parte 2: Variables macroeconómicas


En esta seccción se consolidan y preprocesan las variables macroeconómicas. Los archivos que contienen los valores de estas variables macroeconómicas se guardan en la carpeta /PreparacionDatos/MacroVars. Estos archivos fueron extraídos de las páginas del Banco de la República y del DANE.

In [10]:
### Se crea una lista de years, un diccionario para los cruces y un diccionario para cada year
dic_cruces = {}
years = [2016, 2017, 2018]
years_aux = [2015, 2016, 2017, 2018]   ### Estos son para calcular variacion porcentual
for y in years:
    dic_cruces[y] = {}

##### TRM

In [11]:

### Se toma la TRM anualizada
### Se calcula la variación porcentual anual para esta variable
    
trm = pd.read_excel('MacroVars/trm.xlsx', skiprows=7)
lista_aux = []
for y in years_aux:
    lista_aux.append(trm['Annual Average Market Exchange Rate (Colombian pesos)'][trm['Year']==y].values[0])

voyvo = 0
for y in years:
    anterior = lista_aux[voyvo]
    actual = lista_aux[voyvo+1]
    dic_cruces[y]['TRM'] =  (actual-anterior)/abs(anterior)
    voyvo = voyvo+1


###### PIB

In [12]:
### Se toma la columna que representa la varación porcentual anual del PIB
### (no es necesario que nosotros la transformemos)
    
pib = pd.read_excel('MacroVars/pib.xls', skiprows=4)
## La primera columna es el año.
pib = pib.rename(columns={pib.columns[0]: "Year"})

for y in years:
    ### Correccion para el de 2018
    estepib = y
    if estepib == 2018:
        estepib='2018 (p)'
    dic_cruces[y]['PIB'] = pib['Variación porcentual'][pib['Year']==estepib].values[0]/100


##### Desempleo

In [13]:
### Aqui lo que se tiene es la tasa de desempleo mensual
### Para anualizar esta tasa, usaremos la aproximacion de usar el promedio 
### de todos los meses de cada year
    
### Esta variable se deja como la tasa anual (no se lleva a variación porcentual,
### se decide dejarla como la tasa la cual ya representa un porcentaje)
    
desempleo = pd.read_excel('MacroVars/desempleo.xlsx', skiprows=8, nrows=225) 
desempleo['Year'] = desempleo['Año(aaaa)-Mes(mm)'].str[:4]
for y in years:
    dic_cruces[y]['Desempleo'] = desempleo['Tasa de desempleo (%)'][desempleo['Year']==str(y)].mean() / 100


##### Inflación

In [14]:
### Aqui basta simplemente con tomar la inflacion de año corrido en diciembre
### de cada año. Se divide sobre 100 para que quede en escala real y no con %.
    
### Esta variable ya de por sí representaría una variación porcentual en los precios
### (no es necesario que nosotros la transformemos)
    
inflacion = pd.read_excel('MacroVars/inflacion.xlsx', skiprows=6)
inflacion.index=inflacion['Mes']
for y in years:
    dic_cruces[y]['Inflacion'] = inflacion[y]['En año corrido']/100 

##### Tasa de intervención

In [15]:
### Aqui lo que se tiene es la tasa de intervencion diaria (solo para dias habiles)
### Para anualizar esta tasa, usaremos la aproximacion de usar el promedio 
### de todos los dias de cada year en los que hay registro (todos los dias habiles)

### Esta variable se deja como la tasa anual (no se lleva a variación porcentual,
### se decide dejarla como la tasa la cual ya representa un porcentaje)

tasa_interv = pd.read_excel('MacroVars/tasa_intervencion_diaria.xlsx', skiprows=7, nrows=5000) 
tasa_interv['Year'] = tasa_interv['Fecha (dd/mm/aaaa)'].dt.year


lista_aux = []
for y in years_aux:
    lista_aux.append(tasa_interv['Tasa de intervención de política monetaria'][tasa_interv['Year']==y].mean())

voyvo = 0
for y in years:
    anterior = lista_aux[voyvo]
    actual = lista_aux[voyvo+1]
#    dic_cruces[y]['Tasa_Intervencion'] =  (actual-anterior)/abs(anterior)   ### Como variacion porcentual
    dic_cruces[y]['Tasa_Intervencion'] =  actual  ### Como la tasa de cada year
    voyvo = voyvo+1



##### Balance en Cuenta Corriente

In [16]:
### Usar la columna de cada año, en la fila que tiene el valor para cuenta corriente
### Se calcula la variación porcentual anual para esta variable
    
bal_cc = pd.read_excel('MacroVars/balance_cc.xlsx', skiprows=10)
estos_years =  ['2015 (r)', '2016 (r)', '2017 (pr)', '2018 (pr)']
voyen = 0
bal_cc.index=bal_cc['Cuenta']
for y in years:
    anterior = bal_cc[estos_years[voyen]]['1 Cuenta corriente']
    actual = bal_cc[estos_years[voyen+1]]['1 Cuenta corriente']
    dic_cruces[y]['Balance_CC'] = (actual-anterior)/abs(anterior)
    voyen = voyen+1
    

##### Balance Fiscal

In [17]:
### Usar la columna 'DEFICIT (-) O SUPERAVIT (+)' como el balance_fiscal para cada año.
### Se calcula la variación porcentual anual para esta variable    

bal_fiscal = pd.read_excel('MacroVars/balance_fiscal.xls', skiprows=4)
bal_fiscal = bal_fiscal.rename(columns={bal_fiscal.columns[1]: "Year"})
estos_years =  ['2015 (pr)', '2016 (pr)', '2017 (pr)', '2018 (pr)']
voyen = 0
for y in years:
    anterior = bal_fiscal['DEFICIT (-) O SUPERAVIT (+)'][bal_fiscal['Year']==estos_years[voyen]].values[0]
    actual = bal_fiscal['DEFICIT (-) O SUPERAVIT (+)'][bal_fiscal['Year']==estos_years[voyen+1]].values[0]
    dic_cruces[y]['Balance_Fiscal'] = (actual-anterior)/abs(anterior)
    voyen = voyen+1
    
    

### Parte 3: cruzar variables de salida y variables explicativa para generar el dataset consolidado

In [18]:
### Leer el de salidas
salidas_son = pd.read_csv('Auxi/Costos_Gastos_ventas.csv')

In [19]:
### Construccion de dataframe completa
data_full = salidas_son.copy()


In [20]:
### Nombres de las variables
vars_son = list(dic_cruces[2016].keys())  ### Nombres de variables (son iguales para todos los años)


In [21]:
### Hacer los cruces
for v in vars_son:
    lista_v = []
    for y in salidas_son['Year']:
        lista_v.append(dic_cruces[y][v])
    data_full[v] = lista_v


###### Guardar el dataframe final consolidado.

In [22]:
data_full.to_csv('Datos_completos.csv', index=None)

El archivo "Datos_completos.csv" es el que es usado por el programa de R (es el archivo que el programa de R lee al inicio).